In [1]:
import numpy as np
import pandas as pd
import mglearn
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
from matplotlib import font_manager, rc
import seaborn as sns

D:\anaconda\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
x = pd.read_csv("variable_total.csv")

In [3]:
x = x.drop(["Unnamed: 0"],axis=1)

In [4]:
x.shape

(1997, 79)

In [5]:
y_onehot = pd.read_csv("train_profile_onehot.csv")

In [6]:
y_onehot

,CUS_ID,F20-,F30,F40+,M20-,M30,M40+
0,1,0,0,0,0,0,1
1,2,1,0,0,0,0,0
2,4,0,0,0,0,1,0
3,5,0,0,0,0,0,1
4,8,0,0,0,0,1,0
...,...,...,...,...,...,...,...
1992,2494,0,0,0,1,0,0
1993,2495,0,1,0,0,0,0
1994,2496,0,0,0,0,0,1
1995,2498,0,0,1,0,0,0


In [7]:
y = pd.read_csv("train_profile_group.csv")

In [8]:
y

,CUS_ID,GROUP
0,1,5
1,2,0
2,4,4
3,5,5
4,8,4
...,...,...
1992,2494,3
1993,2495,1
1994,2496,5
1995,2498,2


---
train test 나누기 <br>
minmaxscaled

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
y=y.iloc[:,1]

In [11]:
y_list=y.values.tolist()

In [12]:
y_list=np.array(y_list)

In [13]:
y_list

array([5, 0, 4, ..., 5, 2, 3])

-----
원핫 한거 split

In [14]:
y_onehot=y_onehot.iloc[:,1:7]

In [15]:
y_list_one = y_onehot.values.tolist()

In [16]:
y_list_one = np.array(y_list_one)

In [17]:
y_list_one

array([[0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [18]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

------
knn

In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
pipe = make_pipeline(MinMaxScaler(),KNeighborsClassifier())

In [26]:
pipe.steps

[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
 ('kneighborsclassifier',
  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                       metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                       weights='uniform'))]

In [27]:
param_grid = {'kneighborsclassifier__n_neighbors':[1,3,5,7,9]}

In [36]:
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
grid.best_score_

0.26893859649122803

In [35]:
grid.score(x,y_list)

0.4641962944416625

In [69]:
grid.best_params_

{'ridge__alpha': 10}

In [41]:
pipe1 = make_pipeline(MinMaxScaler(),KNeighborsClassifier(n_neighbors=9))

In [42]:
pipe1.fit(x,y_list)

Pipeline(memory=None,
         steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('kneighborsclassifier',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=9, p=2,
                                      weights='uniform'))],
         verbose=False)

In [43]:
pipe1.score(x,y_list)

0.4641962944416625

In [46]:
# 원핫 한거
grid.fit(x,y_list_one)
grid.best_score_

0.2514010025062656

In [48]:
grid.score(x,y_list_one)

1.0

In [49]:
grid.best_params_

{'kneighborsclassifier__n_neighbors': 1}

In [50]:
# scaled 없이
clf = KNeighborsClassifier(n_neighbors=9)

In [51]:
clf.fit(x,y_list)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [53]:
clf.score(x,y_list)

0.4071106659989985

In [54]:
# scaled 없이 원핫 o
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(x,y_list_one)
clf.score(x,y_list_one)

1.0

In [55]:
from sklearn.neighbors import KNeighborsRegressor

In [56]:
pipe = make_pipeline(MinMaxScaler(),KNeighborsRegressor())
param_grid = {'kneighborsregressor__n_neighbors':[1,3,5,7,9]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

0.010799118730401914
0.20773585580501083
{'kneighborsregressor__n_neighbors': 9}


In [57]:
# scaled 없이
clf = KNeighborsRegressor(n_neighbors=9)
clf.fit(x,y_list)
clf.score(x,y_list)

0.1478519021691086

In [58]:
# onehot o
grid.fit(x,y_list_one)
print(grid.best_score_)
print(grid.score(x,y_list_one))
print(grid.best_params_)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_av

-0.05726745081813257
0.16833021130302805
{'kneighborsregressor__n_neighbors': 9}


D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [59]:
# scaled 없이 원핫 o
clf = KNeighborsRegressor(n_neighbors=9)
clf.fit(x,y_list_one)
clf.score(x,y_list_one)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.12190501574737425

----
선형

In [60]:
from sklearn.linear_model import LinearRegression

In [61]:
pipe = make_pipeline(MinMaxScaler(),LinearRegression())
pipe.fit(x,y_list)
print(pipe.score(x,y_list))

0.21010804099284874


In [62]:
aa = LinearRegression()
aa.fit(x,y_list)
aa.score(x,y_list)

0.2101066505778251

In [64]:
# onehot o
pipe.fit(x,y_list_one)
print(pipe.score(x,y_list_one))

0.11607566097901686


D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [65]:
# scaled 없이 원핫 o
aa = LinearRegression()
aa.fit(x,y_list_one)
aa.score(x,y_list_one)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.11608023185938524

---
리지 회귀

In [66]:
from sklearn.linear_model import Ridge

In [67]:
pipe = make_pipeline(MinMaxScaler(),Ridge())
param_grid = {'ridge__alpha':[0.01,0.1,1,10,100]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

0.13151841892989552
0.17130187423825072
{'ridge__alpha': 10}


In [70]:
aa = Ridge(alpha=10)
aa.fit(x,y_list)
aa.score(x,y_list)

0.21010665057766664

In [72]:
grid.fit(x,y_list_one)
print(grid.best_score_)
print(grid.score(x,y_list_one))
print(grid.best_params_)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_av

0.05304455794762879
0.08666404382052913
{'ridge__alpha': 10}


D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [73]:
aa.fit(x,y_list_one)
aa.score(x,y_list_one)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.11608023185925997

----
라소 회귀

In [74]:
from sklearn.linear_model import Lasso

In [76]:
pipe = make_pipeline(MinMaxScaler(),Lasso(max_iter=100000))
param_grid = {'lasso__alpha':[0.001,0.01,0.1,1,10,100]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

0.11649370626764058
0.19685382699179133
{'lasso__alpha': 0.001}


In [77]:
aa = Lasso(alpha=0.001,max_iter=100000)
aa.fit(x,y_list)
aa.score(x,y_list)

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2314.793091622307, tolerance: 0.592689534301452
  positive)


0.21010664722903005

In [78]:
grid.fit(x,y_list_one)
print(grid.best_score_)
print(grid.score(x,y_list_one))
print(grid.best_params_)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_av

0.0474664395228652
0.07021096149204185
{'lasso__alpha': 0.001}


D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_av

In [79]:
aa = Lasso(alpha=0.001,max_iter=100000)

In [80]:
aa.fit(x,y_list_one)
aa.score(x,y_list_one)

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 168.45425264741854, tolerance: 0.041731697546319516
  positive)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.11608014667289134

----
로지스틱 회귀

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [82]:
pipe = make_pipeline(MinMaxScaler(),LogisticRegression(max_iter=100000))
param_grid = {'logisticregression__C':[0.001,0.01,0.1,1,10,100]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

0.37254260651629073
0.42213319979969954
{'logisticregression__C': 1}


In [84]:
aa = LogisticRegression(C=1,max_iter=100000)
aa.fit(x,y_list)
aa.score(x,y_list)

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.44767150726089133

----
linearSVC

In [85]:
pipe = make_pipeline(MinMaxScaler(),LinearSVC(max_iter=100000))
param_grid = {'linearsvc__C':[0.001,0.01,0.1,1,10,100]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

0.3680501253132832
0.44366549824737106
{'linearsvc__C': 100}


In [86]:
aa = LinearSVC(C=100,max_iter=100000)
aa.fit(x,y_list)
aa.score(x,y_list)

D:\anaconda\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.29894842263395094

---
결정 트리

In [3]:
from sklearn.tree import DecisionTreeClassifier

In [88]:
pipe = make_pipeline(MinMaxScaler(),DecisionTreeClassifier())
param_grid = {'decisiontreeclassifier__max_depth':[4,6,8,10,12,14,16]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

0.2989436090225564
0.9764646970455684
{'decisiontreeclassifier__max_depth': 14}


In [90]:
aa = DecisionTreeClassifier(max_depth=14)
aa.fit(x,y_list)
aa.score(x,y_list)

0.9764646970455684

In [91]:
grid.fit(x,y_list_one)
print(grid.best_score_)
print(grid.score(x,y_list_one))
print(grid.best_params_)

0.2714022556390977
0.9218828242363545
{'decisiontreeclassifier__max_depth': 12}


In [23]:
aa = DecisionTreeClassifier(max_depth=12)
aa.fit(x,y_list_one)
aa.score(x,y_list_one)

0.9218828242363545

In [24]:
aa.feature_importances_

array([0.0088396 , 0.02205481, 0.00867662, 0.02108649, 0.02614728,
       0.0206795 , 0.00644635, 0.00726735, 0.01807899, 0.01896425,
       0.00812934, 0.04030353, 0.01069052, 0.01916442, 0.02084437,
       0.00867836, 0.00961672, 0.00603158, 0.02358626, 0.02404421,
       0.02994439, 0.01548118, 0.00388864, 0.00945186, 0.00603844,
       0.00818754, 0.00156349, 0.00815544, 0.0189822 , 0.00963241,
       0.01362475, 0.00863252, 0.01386057, 0.01040823, 0.00277735,
       0.01117521, 0.0110182 , 0.01705472, 0.01023909, 0.00770279,
       0.00793518, 0.00753301, 0.00768846, 0.00883613, 0.0081729 ,
       0.01426546, 0.00660804, 0.00918906, 0.01003506, 0.01381804,
       0.00610862, 0.02841968, 0.01716101, 0.01346116, 0.01556091,
       0.01056234, 0.0061889 , 0.01172433, 0.009466  , 0.01012707,
       0.00615746, 0.01368451, 0.01009492, 0.01036873, 0.02210487,
       0.0081758 , 0.01323602, 0.01412662, 0.01812671, 0.00692362,
       0.00732085, 0.01207154, 0.01275466, 0.01044813, 0.01333

In [25]:
a=x.columns.tolist()

In [26]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'Malgun Gothic'

In [ ]:
def plot_feature_importances_cancer(model):
    n_features = x.shape[1]
    plt.barh(np.arange(n_features), model.feature_importances_,align='center')
    plt.yticks(np.arange(n_features), a)
    plt.xlabel("특성중요도")
    plt.ylabel("특성")
    plt.ylim(-1,n_features)
    plt.rcParams["figure.figsize"] = (30,100)
plot_feature_importances_cancer(aa)

----
랜덤 포레스트

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
pipe = make_pipeline(MinMaxScaler(),RandomForestClassifier())
param_grid = {'randomforestclassifier__n_estimators':[10,50,100],
             'randomforestclassifier__max_features':[4,6,8,10,12,14,16]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

0.409110275689223
1.0
{'randomforestclassifier__max_features': 16, 'randomforestclassifier__n_estimators': 100}


In [23]:
 aa = RandomForestClassifier(max_features=16, n_estimators=100)
aa.fit(x,y_list)
aa.score(x,y_list)

1.0

In [24]:
grid.fit(x,y_list_one)
print(grid.best_score_)
print(grid.score(x,y_list_one))
print(grid.best_params_)

0.08363157894736842
0.8768152228342514
{'randomforestclassifier__max_features': 16, 'randomforestclassifier__n_estimators': 10}


In [27]:
aa = RandomForestClassifier(max_features=16, n_estimators=10)
aa.fit(x,y_list_one)
aa.score(x,y_list_one)

0.8738107160741112

---------------
그레디언트 부스팅 회귀 트리

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

In [29]:
pipe = make_pipeline(MinMaxScaler(), GradientBoostingClassifier())
param_grid = {'gradientboostingclassifier__n_estimators':[10,50,100],
             'gradientboostingclassifier__max_features':[4,6,8,10,12,14,16],
            'gradientboostingclassifier__learning_rate':[0.1,0.01,0.001,0.0001] }
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

0.3955927318295739
0.6730095142714071
{'gradientboostingclassifier__learning_rate': 0.1, 'gradientboostingclassifier__max_features': 4, 'gradientboostingclassifier__n_estimators': 50}


In [31]:
 aa = GradientBoostingClassifier(max_features=4, n_estimators=50,learning_rate=0.1)
aa.fit(x,y_list)
aa.score(x,y_list)

0.6875312969454181

----
선형 svc

In [33]:
from sklearn.svm import LinearSVC

In [34]:
pipe = make_pipeline(MinMaxScaler(), LinearSVC())
pipe.fit(x,y_list)
print(pipe.score(x,y_list))

0.4296444667000501


In [35]:
 aa = LinearSVC()
aa.fit(x,y_list)
aa.score(x,y_list)

D:\anaconda\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.14872308462694042

---
서포트 백터 머신 (SVM)

In [37]:
from sklearn.svm import SVC

In [38]:
pipe = make_pipeline(MinMaxScaler(), SVC())
param_grid = {'svc__C':[0.1,10,100,1000],
             'svc__gamma':[0.1,10,100,1000],}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

0.3665538847117794
0.44917376064096143
{'svc__C': 10, 'svc__gamma': 0.1}


In [43]:
aa = SVC(C=10, gamma=0.1)
aa.fit(x,y_list)
aa.score(x,y_list)

1.0

---
신경망 

In [44]:
from sklearn.neural_network import MLPClassifier

In [49]:
pipe = make_pipeline(MinMaxScaler(), MLPClassifier(solver='adam'))
param_grid = {'mlpclassifier__alpha':[0.001,0.01,0.1,10,100],}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x,y_list)
print(grid.best_score_)
print(grid.score(x,y_list))
print(grid.best_params_)

D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\anaconda\lib\site-packages\sklear

0.3690664160401002
0.5833750625938908
{'mlpclassifier__alpha': 0.01}


D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [50]:
aa = MLPClassifier(solver='adam', alpha =0.01)
aa.fit(x,y_list)
aa.score(x,y_list)

D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.45818728092138206

In [51]:
grid.fit(x,y_list_one)
print(grid.best_score_)
print(grid.score(x,y_list_one))
print(grid.best_params_)

D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\anaconda\lib\site-packages\sklear

0.146734335839599
0.24086129193790687
{'mlpclassifier__alpha': 0.01}


D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [53]:
aa = MLPClassifier(solver='adam', alpha =0.01)
aa.fit(x,y_list_one)
aa.score(x,y_list_one)

D:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.24837255883825737